# Limpieza y Preprocesado

In [85]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer



df = pd.read_csv("../data/housing.csv")


Aquí Realizaremos la inputación de mediana a aquellos valores faltantes para no perder información y reemplazarlos. Como se muestra ya no hay nulos.

In [40]:
df = df.drop_duplicates()
df["total_bedrooms"].fillna(df["total_bedrooms"].median())   # Aquí elegí inputar datos faltantes a removerlos
df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Aquí prepararemos los conjuntos de entrenamiento y prueba

In [41]:
X = df.drop("median_house_value", axis=1)
y = df["median_house_value"]

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Posteriormente, se construirá un pipeline de preprocesamiento en el que primero se separarán las variables numéricas y categóricas para aplicarles transformaciones específicas.

A las variables numéricas se les incorporará la imputación por mediana (incluida dentro del pipeline como buena práctica), se aplicará transformación logarítmica a aquellas con asimetría positiva identificada en el EDA y finalmente se realizará el escalamiento para estandarizar su magnitud.

En el caso de las variables categóricas, se llevará a cabo la imputación de valores faltantes y su codificación mediante One Hot Encoding para representarlas en formato numérico.

In [99]:
numeric_features = [
    "longitude",
    "latitude",
    "housing_median_age",
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income"
]
categorical_features = ["ocean_proximity"]

log_features = [
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income"
]

num_no_log = list(set(numeric_features) - set(log_features))

log_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("log_transform", FunctionTransformer(np.log1p, feature_names_out="one-to-one")),
    ("scaler", StandardScaler())
])

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessing_pipeline = ColumnTransformer([
    ("num_log", log_pipeline, log_features),
    ("num", num_pipeline, num_no_log),
    ("cat", cat_pipeline, categorical_features)
])


In [106]:
X_train_prepared = preprocessing_pipeline.fit_transform(X_train)
X_test_prepared = preprocessing_pipeline.transform(X_test)

In [110]:
import re    # esta función la incluyo para limpiar cualquier bracket que haya quedado en el encoding al pasarlo a df
def clean_col_names(df):
    df.columns = [re.sub(r"[\[\]<>]", "", col) for col in df.columns]
    return df

In [111]:
col_names = preprocessing_pipeline.get_feature_names_out()

X_train_prepared = clean_col_names(pd.DataFrame(X_train_prepared, columns=col_names))
X_test_prepared  = clean_col_names(pd.DataFrame(X_test_prepared,  columns=col_names))


# guardo los conjuntos de datos para ocuparlos en el script del modelo
X_train_prepared.to_parquet("../data/X_train_prepared.parquet", index=False)
X_test_prepared.to_parquet("../data/X_test_prepared.parquet",  index=False)

pd.Series(y_train, name="median_house_value").to_frame().to_parquet("../data/y_train.parquet", index=False)
pd.Series(y_test,  name="median_house_value").to_frame().to_parquet("../data/y_test.parquet",  index=False)